# Decrypt datasets

Here is where **attestation** takes place. We need the key to decrypt the datasets, but it is stored in a remote attester (Trustee), and it will be provided to us only if **attestation** is successful, meaning the software & hardware of the CVM have not been tampered with.

By doing this we ensure that the CVM is **safe** and having the right hardware and software running prevents any attacker from fetching the transactions while they are being read by the model (**data in use**). This is possible because the hardware inside the CVM makes sure that all data being loaded in the memory is encrypted, so that if an attacker tries to do a physical/virtual memory dump, the output will only be encrypted/zeroed blobs of memory.

This is what **Confidential Computing** is about: securing data in use.

Let's start by fetching the required key:

In [ ]:
%%bash

TRUSTEE_KEY_LOCATION=/home/azureuser/test/tee_key.pem
TRUSTEE_IP=10.0.0.4
ATTESTATION_TOKEN=./attestation_token

TRUESTEE_KEY_PATH=default/fraud-detection/data_key

echo "Attestation..."
sudo /usr/local/bin/kbs-client --url http://$TRUSTEE_IP:8080 attest --tee-key-file $TRUSTEE_KEY_LOCATION > $ATTESTATION_TOKEN
if [ ! -s $ATTESTATION_TOKEN ]; then
  echo "ERROR: Attestation token is empty!"
  exit 1
fi

echo ""
echo "Fetching key..."
sudo /usr/local/bin/kbs-client --url http://$TRUSTEE_IP:8080 get-resource --attestation-token $ATTESTATION_TOKEN --tee-key-file $TRUSTEE_KEY_LOCATION --path $TRUESTEE_KEY_PATH > base64_out
if [ ! -s base64_out ]; then
  echo "ERROR: No key retourned!"
  exit 1
fi
DECODED=$(base64 --decode < base64_out)
rm -rf base64_out

mkdir -p keys

echo "keys:"
ls keys

echo ""
if [ -z "$DECODED" ]; then
    echo "Key fetch failure!"
    exit 1
fi
echo "Key fetched!"
echo "$DECODED" > keys/key.bin
echo ""

echo "keys:"
ls keys

rm -f $ATTESTATION_TOKEN

Since we got a key, let's now decrypt the models!

In [ ]:
%%bash

KEY_FILE=keys/key.bin
DATASET_SRC=downloaded_datasets
DATASET_DEST=datasets_dec

mkdir -p $DATASET_DEST
rm -rf $DATASET_DEST/*

for file in $DATASET_SRC/*; do
    fname=$(basename $file)
    fname=${fname%.enc}
    openssl enc -d -aes-256-cfb -pbkdf2 -kfile $KEY_FILE -in $file -out $DATASET_DEST/$fname
    echo "Decrypted" $DATASET_DEST/$fname":"
    head $DATASET_DEST/$fname
    echo ""
done

ls $DATASET_DEST

Get rid of the key, since we don't need it anymore. The key is anyways stored in an attested CVM (so no intruder can enter), and stored in an encrypted disk.

In [ ]:
! rm -rf keys/*